# CNN #

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv
import time

I started out by first defining a number of functions that would be necessary to training models. I also worked on splitting up and sanitizing the data. Additionally, I generated some dummy data in the shape of my original data to simply prove that my CNN was properly set up to learn.  I tested my CNN using this data and demonstrated that it was in fact working.

In [3]:
nb_classes = 2


def print_confusion_matric(nb_classes, dataloader):
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(dataloader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs.float())
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    print(confusion_matrix)
    new_confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i, row in enumerate(confusion_matrix):
        sum = row.sum()
        new_confusion_matrix[i, 0] = 100 * row[0] / sum
        new_confusion_matrix[i, 1] = 100 * row[1] / sum
    conf_matrix = pd.DataFrame(
        new_confusion_matrix.numpy(),
        index=["open", "closed"],
        columns=["open", "closed"],
    )
    # cf.style.background_gradient(cmap='Blues').format("{:.1f} %")
    print("Percent Correct")
    return conf_matrix.style.background_gradient(cmap="Blues").format("{:.1f}")
    # print(confusion_matrix)

In [4]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"
fp_csv_file = "EEG_Eye_State_Train.csv"
# fp_csv_file = "Dummy_Train.csv"
eeg_data = pd.read_csv(
    os.path.join(root_directory, fp_csv_file), delimiter=",", usecols=range(15)
).dropna(axis=0)

# print(eeg_data.iloc[0, :])
open_or_closed = eeg_data.iloc[0, 14]
channel_data = eeg_data.iloc[0, 0:14]
channel_data = np.array([channel_data])
channel_data = channel_data.astype("float").reshape(-1, 14)
# print(open_or_closed)
# print(channel_data)
eeg_data.shape

(11984, 15)

In [5]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Train.csv"
validate_file = "EEG_Eye_State_Validate.csv"
test_file = "EEG_Eye_State_Test.csv"

# train_file = "Dummy_Train.csv"
# validate_file = "Dummy_Validate.csv"
# test_file = "Dummy_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [7]:
# class ConvNet(torch.nn.Module):
#     def __init__(
#         self,
#         n_inputs,
#         n_hiddens_per_conv_layer,
#         n_hiddens_per_fc_layer,
#         n_outputs,
#         patch_size_per_conv_layer,
#         stride_per_conv_layer,
#         activation_function="tanh",
#         device="cpu",
#     ):

#         super().__init__()

#         self.device = device

#         n_conv_layers = len(n_hiddens_per_conv_layer)
#         if (
#             len(patch_size_per_conv_layer) != n_conv_layers
#             or len(stride_per_conv_layer) != n_conv_layers
#         ):
#             raise Exception(
#                 "The lengths of n_hiddens_per_conv_layer, patch_size_per_conv_layer, and stride_per_conv_layer must be equal."
#             )

#         self.activation_function = (
#             torch.tanh if activation_function == "tanh" else torch.relu
#         )

#         # Create all convolutional layers
#         # First argument to first Conv1d is number of channels for a sample
#         n_in = 14
#         input_hw = int(
#             np.sqrt(n_inputs)
#         )  # original input image height (=width because image assumed square)
#         self.conv_layers = torch.nn.ModuleList()
#         for nh, patch_size, stride in zip(
#             n_hiddens_per_conv_layer, patch_size_per_conv_layer, stride_per_conv_layer
#         ):
#             self.conv_layers.append(
#                 torch.nn.Conv1d(n_in, nh, kernel_size=patch_size, stride=stride)
#             )
#             conv_layer_output_hw = (input_hw - patch_size) // stride + 1
#             input_hw = conv_layer_output_hw  # for next trip through this loop
#             n_in = nh

#         # Create all fully connected layers.  First must determine number of inputs to first
#         # fully-connected layer that results from flattening the images coming out of the last
#         # convolutional layer.
#         n_in = input_hw ** 2 * n_in  # n_hiddens_per_fc_layer[0]
#         self.fc_layers = torch.nn.ModuleList()
#         for nh in n_hiddens_per_fc_layer:
#             self.fc_layers.append(torch.nn.Linear(n_in, nh))
#             n_in = nh
#         self.fc_layers.append(torch.nn.Linear(n_in, n_outputs))

#         self.to(self.device)

#     def forward_all_outputs(self, X):
#         n_samples = X.shape[0]
#         Ys = [X]
#         for conv_layer in self.conv_layers:
#             Ys.append(self.activation_function(conv_layer(Ys[-1])))

#         for layeri, fc_layer in enumerate(self.fc_layers[:-1]):
#             if layeri == 0:
#                 Ys.append(
#                     self.activation_function(fc_layer(Ys[-1].reshape(n_samples, -1)))
#                 )
#             else:
#                 Ys.append(self.activation_function(fc_layer(Ys[-1])))

#         Ys.append(self.fc_layers[-1](Ys[-1]))
#         return Ys

#     def forward(self, X):
#         Ys = self.forward_all_outputs(X)
#         return Ys[-1]

#     def train(
#         self, X, T, batch_size, n_epochs, learning_rate, method="sgd", verbose=True
#     ):

#         # Set data matrices to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         if not isinstance(T, torch.Tensor):
#             T = (
#                 torch.from_numpy(T).long().to(self.device)
#             )  # required for classification in pytorch

#         X.requires_grad_(True)

#         self.classes = torch.unique(T)

#         if method == "sgd":
#             optimizer = torch.optim.SGD(
#                 self.parameters(), lr=learning_rate, momentum=0.9
#             )
#         else:
#             optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

#         CELoss = torch.nn.CrossEntropyLoss(reduction="mean")
#         self.error_trace = []

#         for epoch in range(n_epochs):

#             num_batches = X.shape[0] // batch_size
#             loss_sum = 0

#             for k in range(num_batches):
#                 start = k * batch_size
#                 end = (k + 1) * batch_size
#                 X_batch = X[start:end, ...]
#                 T_batch = T[start:end, ...]

#                 Y = self.forward(X_batch)

#                 loss = CELoss(Y, T_batch)
#                 loss.backward()

#                 # Update parameters
#                 optimizer.step()
#                 optimizer.zero_grad()

#                 loss_sum += loss

#             self.error_trace.append(loss_sum / num_batches)

#             if n_epochs >= 10:
#                 if verbose and (epoch + 1) % (n_epochs // 10) == 0:
#                     print(
#                         f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}"
#                     )
#             else:
#                 print(f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}")

#         return self

#     def softmax(self, Y):
#         """Apply to final layer weighted sum outputs"""
#         # Trick to avoid overflow
#         maxY = torch.max(Y, axis=1)[0].reshape((-1, 1))
#         expY = torch.exp(Y - maxY)
#         denom = torch.sum(expY, axis=1).reshape((-1, 1))
#         Y = expY / denom
#         return Y

#     def use(self, X):
#         # Set input matrix to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         Y = self.forward(X)
#         probs = self.softmax(Y)
#         classes = self.classes[torch.argmax(probs, axis=1).cpu().numpy()]
#         return classes.cpu().numpy(), probs.detach().cpu().numpy()

In [8]:
# import random


# with open("Dummy_Lumped_Test.csv", "w", newline="") as csvfile:
#     spamwriter = csv.writer(
#         csvfile, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL
#     )
#     col_labels = [
#         "AF3_0",
#         "F7_0",
#         "F3_0",
#         "FC5_0",
#         "T7_0",
#         "P7_0",
#         "O1_0",
#         "O2_0",
#         "P8_0",
#         "T8_0",
#         "FC6_0",
#         "F4_0",
#         "F8_0",
#         "AF4_0",
#         "AF3_1",
#         "F7_1",
#         "F3_1",
#         "FC5_1",
#         "T7_1",
#         "P7_1",
#         "O1_1",
#         "O2_1",
#         "P8_1",
#         "T8_1",
#         "FC6_1",
#         "F4_1",
#         "F8_1",
#         "AF4_1",
#         "AF3_2",
#         "F7_2",
#         "F3_2",
#         "FC5_2",
#         "T7_2",
#         "P7_2",
#         "O1_2",
#         "O2_2",
#         "P8_2",
#         "T8_2",
#         "FC6_2",
#         "F4_2",
#         "F8_2",
#         "AF4_2",
#         "AF3_3",
#         "F7_3",
#         "F3_3",
#         "FC5_3",
#         "T7_3",
#         "P7_3",
#         "O1_3",
#         "O2_3",
#         "P8_3",
#         "T8_3",
#         "FC6_3",
#         "F4_3",
#         "F8_3",
#         "AF4_3",
#         "AF3_4",
#         "F7_4",
#         "F3_4",
#         "FC5_4",
#         "T7_4",
#         "P7_4",
#         "O1_4",
#         "O2_4",
#         "P8_4",
#         "T8_4",
#         "FC6_4",
#         "F4_4",
#         "F8_4",
#         "AF4_4",
#         "AF3_5",
#         "F7_5",
#         "F3_5",
#         "FC5_5",
#         "T7_5",
#         "P7_5",
#         "O1_5",
#         "O2_5",
#         "P8_5",
#         "T8_5",
#         "FC6_5",
#         "F4_5",
#         "F8_5",
#         "AF4_5",
#         "AF3_6",
#         "F7_6",
#         "F3_6",
#         "FC5_6",
#         "T7_6",
#         "P7_6",
#         "O1_6",
#         "O2_6",
#         "P8_6",
#         "T8_6",
#         "FC6_6",
#         "F4_6",
#         "F8_6",
#         "AF4_6",
#         "AF3_7",
#         "F7_7",
#         "F3_7",
#         "FC5_7",
#         "T7_7",
#         "P7_7",
#         "O1_7",
#         "O2_7",
#         "P8_7",
#         "T8_7",
#         "FC6_7",
#         "F4_7",
#         "F8_7",
#         "AF4_7",
#         "AF3_8",
#         "F7_8",
#         "F3_8",
#         "FC5_8",
#         "T7_8",
#         "P7_8",
#         "O1_8",
#         "O2_8",
#         "P8_8",
#         "T8_8",
#         "FC6_8",
#         "F4_8",
#         "F8_8",
#         "AF4_8",
#         "AF3_9",
#         "F7_9",
#         "F3_9",
#         "FC5_9",
#         "T7_9",
#         "P7_9",
#         "O1_9",
#         "O2_9",
#         "P8_9",
#         "T8_9",
#         "FC6_9",
#         "F4_9",
#         "F8_9",
#         "AF4_9",
#         "eyeDetection",
#     ]
#     spamwriter.writerow(col_labels)

#     list_a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
#     list_b = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
#     list_big_a = []
#     list_big_b = []
#     for i in range(10):
#         for j in range(14):
#             list_big_a.append(list_a[j])
#             list_big_b.append(list_b[j])

#     list_big_a.append(0)
#     list_big_b.append(1)
#     for i in range(500):
#         random_number = random.randint(1, 10)
#         if random_number > 5:
#             spamwriter.writerow(list_big_a)
#         else:
#             spamwriter.writerow(list_big_b)

In [9]:
model = fpnn.CNN(
    1,
    14,
    [10, 20],
    [5, 5],
    2,
    kernel_size_per_conv_layer=[3, 3],
    stride_per_conv_layer=[2, 2],
).to(device)
print(model)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(3,), stride=(2,))
    (1): Conv1d(10, 20, kernel_size=(3,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=40, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=5, bias=True)
    (2): Linear(in_features=5, out_features=2, bias=True)
  )
)


In [10]:
# from functools import reduce
# from operator import mul

# print(torch.rand(5, 14))
# X = torch.rand(100, 14, device=device)
# print(X.shape)
# T = torch.randint(0, 2, (100, 1), device=device)
# logits = model(X)
# print(logits)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    # first = True
    for batch, (X, T) in enumerate(dataloader):
        X, T = X.to(device), T.to(device)

        # Compute prediction error
        Y = model(X.float())
        # if first:
        #     first = False
        #     print('Y.shape', Y.shape)
        #     print('T.shape', T.shape)
        #     print('T: ', T[0])
        loss = loss_fn(Y, T.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


#         if batch % 50 == 0:
#             loss, current = loss.item(), batch * len(X)
# print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, T in dataloader:
            X, T = X.to(device), T.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, T.long()).item()
            correct += (pred.argmax(1) == T).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [13]:
def train_for_epochs(
    epochs, train_dataloader, _validate_dataloader, model, loss_fn, optimizer
):
    start = time.time()
    for t in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        if t % 50 == 0:
            print(f"Epoch {t}\n-------------------------------")
            test(_validate_dataloader, model, loss_fn)
    end = time.time()
    print("Done: ", end - start, " seconds")
    test(_validate_dataloader, model, loss_fn)

In [14]:
# fashion_training_data = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )
# fashion_train_dataloader = DataLoader(fashion_training_data, batch_size=64)
# n_samples, n_inputs = train_dataset.__len__(), 28*28
# n_outputs = 1
# n_hiddens = [10, 10]
# model = fpnn.NeuralNetwork(28*28, n_hiddens, 10).to(device)
# print(model)

# learning_rate = 0.5 / (n_samples * n_outputs)  ## Larger learning rate

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# train(fashion_train_dataloader, model, loss_fn, optimizer)

In [15]:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 14])
Shape of y:  torch.Size([64]) torch.float64


I again started out with a dummy run using dummy data that had a trivial pattern which the neural network should easily be able to learn. I did this to simply test that the network was correctly setup.

In [22]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

# train_file = "EEG_Eye_State_Train.csv"
# validate_file = "EEG_Eye_State_Validate.csv"
# test_file = "EEG_Eye_State_Test.csv"

train_file = "Dummy_Lumped_Train.csv"
validate_file = "Dummy_Lumped_Validate.csv"
test_file = "Dummy_Lumped_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [23]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 100
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.069871 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000094 

Done:  11.338661193847656  seconds
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000063 



In [24]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000063 



domain model result.

# SGD Training #

I next began the task of training my model using SGD and various parameters.

In [16]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Lumped_10_Normalized_Train.csv"
validate_file = "EEG_Eye_State_Lumped_10_Normalized_Validate.csv"
test_file = "EEG_Eye_State_Lumped_10_Normalized_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [27]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.603634 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.299650 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.278495 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.282074 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.288265 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.308168 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg

In [28]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.631273 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.207903 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.228532 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.238833 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.245316 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.250075 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg

In [30]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [1000, 20, 10]
num_hiddens_per_fc_layer = [500, 15, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10]
stride_per_conv_layer = [1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 1000, kernel_size=(10,), stride=(1,))
    (1): Conv1d(1000, 20, kernel_size=(10,), stride=(1,))
    (2): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1130, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.608456 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.269878 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.382582 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.410375 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.427347 

Epoch 250
-------------------------------
Test E

In [31]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [10]
num_hiddens_per_fc_layer = [10]
num_outputs = 2
kernel_size_per_conv_layer = [10]
stride_per_conv_layer = [1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1310, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 63.8%, Avg loss: 0.554793 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.407030 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.443656 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.463671 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.470202 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.474084 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.476955 

Epoch 350
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.479734 

Epoc

In [32]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [
    int(np.sqrt(np.prod(117))) * 2,
    int(np.sqrt(np.prod(117))),
    10,
]
num_hiddens_per_fc_layer = [
    int(np.sqrt(np.prod(117))) * 2,
    int(np.sqrt(np.prod(117))),
    10,
]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10]
stride_per_conv_layer = [1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
    (2): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1130, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.707113 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.275624 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.357023 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.402117 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.421686 

Epoch 250
-------------------------------
Test Error: 

In [34]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20, 10, 10, 5]
num_hiddens_per_fc_layer = [20, 15, 15, 10, 5]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10, 10, 10]
stride_per_conv_layer = [1, 1, 1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
    (2): Conv1d(20, 10, kernel_size=(10,), stride=(1,))
    (3): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
    (4): Conv1d(10, 5, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=475, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=15, bias=True)
    (3): Linear(in_features=15, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=5, bias=True)
    (5): Linear(in_features=5, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.699657 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.269158 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.507028 

Epoch 150
----

In [35]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [100, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 100, kernel_size=(10,), stride=(1,))
    (1): Conv1d(100, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.0%, Avg loss: 0.596524 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.268456 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.331914 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.362615 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.382574 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.395071 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, A

In [25]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [5, 5]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(5,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(5,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2640, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.609820 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.380087 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.568353 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.610893 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.636831 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.657693 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg l

In [26]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [2, 2]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(2,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(2,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2760, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 60.3%, Avg loss: 0.581543 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.434153 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.488630 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.513635 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.530793 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.543828 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg l

In [27]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=580, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.632053 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.226699 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.259285 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.276743 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.288602 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.297687 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg 

In [28]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(8,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 57.4%, Avg loss: 0.667778 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.188128 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.190001 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.194126 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.197455 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.200204 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg lo

In [29]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [5, 5]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(5,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(5,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=640, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 57.4%, Avg loss: 0.624722 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.271135 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.286898 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.292333 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.295809 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.298924 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg lo

In [30]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [3, 3]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(3,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(3,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=240, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 55.3%, Avg loss: 0.629295 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.186632 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.219164 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.235632 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 94.3%, Avg loss: 0.246842 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg loss: 0.255386 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 93.6%, Avg 

In [17]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [15, 15]
stride_per_conv_layer = [3, 3]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(15,), stride=(3,))
    (1): Conv1d(20, 20, kernel_size=(15,), stride=(3,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=200, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 58.2%, Avg loss: 0.636926 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.230332 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.250931 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.263029 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.271775 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.278674 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg 

In [18]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8]
stride_per_conv_layer = [3, 3]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(3,))
    (1): Conv1d(20, 20, kernel_size=(8,), stride=(3,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=260, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.621343 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.416077 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.464866 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.488187 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.505285 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.518826 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg lo

In [19]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [4, 4]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(4,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(4,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=120, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.678946 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.491530 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.562854 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.597880 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.621322 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.638956 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg 

In [23]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.673084 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.278593 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.300344 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.314452 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.323212 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.329800 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg

In [33]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [2, 6]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(2,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(6,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2680, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 58.9%, Avg loss: 0.605998 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.424798 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.476897 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.502969 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.534114 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.548953 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg l

In [34]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 4]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(4,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 60.3%, Avg loss: 0.621485 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.351817 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.408163 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.425960 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.442806 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.456714 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg l

In [35]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 3]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2620, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.7%, Avg loss: 0.592895 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.520418 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.515035 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.534545 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.547724 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.557638 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg l

In [36]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [2, 6]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(2,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(6,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2680, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.699075 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.330502 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.514046 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.581749 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.610944 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.638883 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg l

In [37]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 4]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(4,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 51.1%, Avg loss: 0.686635 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.366431 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.402215 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.420616 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.434534 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.442579 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg l

In [38]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 3]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2620, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.687571 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.390854 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.434929 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.454230 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.468583 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.479255 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg l

In [45]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.695393 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.237071 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.252335 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.261487 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.267720 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.272730 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg

# ADAM Training #

Feeling satisfied with my exploration of SGD I next turned to exploring various parameters of Adam.

In [21]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.732714 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.701324 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 53.2%, Avg loss: 0.723460 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.643362 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.668193 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 58.9%, Avg loss: 0.729707 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 53.9%, Avg

In [22]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 46.8%, Avg loss: 0.703712 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.419755 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 80.9%, Avg loss: 0.365125 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.333831 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.380843 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.303366 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg

In [25]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.09
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.777624 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 58.9%, Avg loss: 0.687228 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 52.5%, Avg loss: 0.763620 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 57.4%, Avg loss: 0.649848 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.633083 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.690304 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 61.7%, Avg

In [26]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=580, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.640823 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.386930 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.263740 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.559458 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.716265 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.784324 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg 

In [27]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [3, 3]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(3,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(3,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=240, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 65.2%, Avg loss: 0.606077 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.416518 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.484869 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.524194 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.556749 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.585488 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg 

In [28]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(8,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.572318 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.256651 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.458445 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.623660 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.700912 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.773380 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg lo

In [29]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [15, 15]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(15,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(15,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=500, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.604658 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.308370 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.486614 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.262857 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.298025 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.493326 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg 

In [30]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [1000, 20]
num_hiddens_per_fc_layer = [500, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 1000, kernel_size=(8,), stride=(2,))
    (1): Conv1d(1000, 20, kernel_size=(8,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=600, out_features=500, bias=True)
    (1): Linear(in_features=500, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 56.7%, Avg loss: 0.690791 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.455958 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.413740 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 70.2%, Avg loss: 0.691636 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.649758 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.541956 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 80.1%, 

In [21]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [40, 35, 30, 20, 10]
num_hiddens_per_fc_layer = [40, 35, 30, 20, 10]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8, 8, 8, 8]
stride_per_conv_layer = [1, 1, 1, 1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 40, kernel_size=(8,), stride=(1,))
    (1): Conv1d(40, 35, kernel_size=(8,), stride=(1,))
    (2): Conv1d(35, 30, kernel_size=(8,), stride=(1,))
    (3): Conv1d(30, 20, kernel_size=(8,), stride=(1,))
    (4): Conv1d(20, 10, kernel_size=(8,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1050, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=35, bias=True)
    (2): Linear(in_features=35, out_features=30, bias=True)
    (3): Linear(in_features=30, out_features=20, bias=True)
    (4): Linear(in_features=20, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 58.9%, Avg loss: 0.665533 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 50.4%, Avg loss: 0.679792 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 52.5%, Avg loss: 0.676287 

Epoch 150
-----

In [24]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20, 10]
num_hiddens_per_fc_layer = [20, 15, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10, 10]
stride_per_conv_layer = [2, 2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(2,))
    (2): Conv1d(20, 10, kernel_size=(10,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=100, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.630838 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.267274 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.374399 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.314057 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.328107 

Epoch 250
-------------------------------
Test Error: 


In [26]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=580, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 62.4%, Avg loss: 0.640823 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.386930 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.263740 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.559458 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.716265 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.784324 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg 

In [26]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [10, 10]
num_hiddens_per_fc_layer = [10, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(10,), stride=(2,))
    (1): Conv1d(10, 10, kernel_size=(10,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=290, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.611366 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.441681 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.423135 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.539528 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.619664 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.672372 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg 

In [27]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [10, 10]
num_hiddens_per_fc_layer = [10, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [3, 3]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(10,), stride=(3,))
    (1): Conv1d(10, 10, kernel_size=(10,), stride=(3,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=120, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 64.5%, Avg loss: 0.586599 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.502980 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.626502 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.690082 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.738267 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.778674 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg 

In [28]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [10, 10]
num_hiddens_per_fc_layer = [10, 10]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 10, kernel_size=(10,), stride=(1,))
    (1): Conv1d(10, 10, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=1220, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 63.1%, Avg loss: 0.634285 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.379583 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.414401 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.405938 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.233407 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.414209 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg

In [29]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 8]
stride_per_conv_layer = [2, 2]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(2,))
    (1): Conv1d(20, 20, kernel_size=(8,), stride=(2,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.0%, Avg loss: 0.613987 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.413713 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.506070 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.546669 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.575430 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.598823 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg lo

In [30]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [4, 4]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(4,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(4,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2680, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 59.6%, Avg loss: 0.632355 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.534352 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.620170 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.662207 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.977704 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 1.250652 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg l

In [31]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [5, 5]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(5,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(5,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2640, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.604107 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.752595 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.814504 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.902324 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.946726 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.993681 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg l

In [32]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [5, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(5,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2540, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 63.1%, Avg loss: 0.655905 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 86.5%, Avg loss: 1.367036 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 87.9%, Avg loss: 1.701217 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 1.905897 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 2.071225 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 2.208817 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg 

In [39]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [2, 6]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(2,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(6,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2680, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 70.9%, Avg loss: 0.522409 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.517982 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.507886 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.591377 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.653872 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg loss: 0.704747 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.9%, Avg l

In [40]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 4]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(4,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 63.8%, Avg loss: 0.588402 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.414646 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.575060 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.635391 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.678324 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.712147 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg l

In [41]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 3]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2620, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 67.4%, Avg loss: 0.594482 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.466382 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.581551 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.674890 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.719178 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.749977 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 92.2%, Avg l

In [42]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [2, 6]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(2,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(6,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2680, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 57.4%, Avg loss: 0.669675 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.431167 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.421917 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.445662 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.432574 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.452535 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg l

In [43]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 4]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(4,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2600, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 61.0%, Avg loss: 0.657329 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.382395 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.285601 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.367531 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.376159 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.588992 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 83.0%, Avg l

In [44]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [8, 3]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(8,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(3,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2620, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 66.0%, Avg loss: 0.566441 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 78.7%, Avg loss: 0.423833 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.326546 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 85.8%, Avg loss: 0.304906 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 89.4%, Avg loss: 0.237043 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.450474 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 85.1%, Avg l

# Final Model Choice #

In [45]:
num_channels = 1
num_inputs = 140
num_hiddens_per_conv_layer = [20, 20]
num_hiddens_per_fc_layer = [20, 15]
num_outputs = 2
kernel_size_per_conv_layer = [10, 10]
stride_per_conv_layer = [1, 1]


model = fpnn.CNN(
    num_channels,
    num_inputs,
    num_hiddens_per_conv_layer,
    num_hiddens_per_fc_layer,
    num_outputs,
    kernel_size_per_conv_layer=kernel_size_per_conv_layer,
    stride_per_conv_layer=stride_per_conv_layer,
    activation_function="relu",
).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.05
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 1000
train_for_epochs(
    epochs, train_dataloader, validate_dataloader, model, loss_fn, optimizer
)

CNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 20, kernel_size=(10,), stride=(1,))
    (1): Conv1d(20, 20, kernel_size=(10,), stride=(1,))
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=2440, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=15, bias=True)
    (2): Linear(in_features=15, out_features=2, bias=True)
  )
)
Epoch 0
-------------------------------
Test Error: 
 Accuracy: 49.6%, Avg loss: 0.695393 

Epoch 50
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.237071 

Epoch 100
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.252335 

Epoch 150
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.261487 

Epoch 200
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.267720 

Epoch 250
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg loss: 0.272730 

Epoch 300
-------------------------------
Test Error: 
 Accuracy: 95.0%, Avg

In [53]:
torch.save(
    model.state_dict(),
    "D:\College\CS_545\Final_Project\Feudale_Final_Project_CNN_Model.pt",
)

In [47]:
test(train_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000001 



In [49]:
test(validate_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 95.0%, Avg loss: 0.308357 



In [50]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 97.2%, Avg loss: 0.094417 



In [51]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs.float())
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)
new_confusion_matrix = torch.zeros(nb_classes, nb_classes)
for i, row in enumerate(confusion_matrix):
    sum = row.sum()
    new_confusion_matrix[i, 0] = 100 * row[0] / sum
    new_confusion_matrix[i, 1] = 100 * row[1] / sum
conf_matrix = pd.DataFrame(
    new_confusion_matrix.numpy(), index=["open", "closed"], columns=["open", "closed"]
)
# cf.style.background_gradient(cmap='Blues').format("{:.1f} %")
print("Percent Correct")
conf_matrix.style.background_gradient(cmap="Blues").format("{:.1f}")
# print(confusion_matrix)

tensor([[82.,  3.],
        [ 1., 55.]])
Percent Correct


,open,closed
open,96.5,3.5
closed,1.8,98.2


After thoroughly exploring numerous parameters I finally settled on using a model with the following parameters:
I had two convolutional hidden layers with 20 units each.  Each convolution layer used a kernel 10 units in length with a stride length of one and a padding size of 0. I also had two fully connected layers with 20 and 15 layers respectively. These came after the convolutional layers in order.
I utilized a learning rate of 0.05. I used SGD as the optimizer function with a momentum of 0.9. I utilized Cross Entropy Loss. I trained this model for 1000 epochs.</br></br>
This model had produced the highest validation accuracy of any model I trained. I evaluated its performance on the train, validation, and test data sets with 100%, 95.0%, and 97.2% accuracies achieved on each data set respectively. When I broke the results out in a confusion matrix as open versus closed my model was correctly identifying cases of open eyes 96.5% of the time and closed eyes 98.2% of the time. This was a fairly even split and showed that the model was not using a strategy biased in favor of one State versus the other.  Overall, I was very satisfied with these results.